# Questão 1

Classes:
    - Promotor
    - Detrator
    - Neutro
Só 100 notícias são classificadas
Os conteúdos são rankeados por alcance
    - No grupo com os 50 maiores alcances, todos são classificados manualmente
    - O restante, 50 são selecionados aleatoriamentes e classificados manualmente
    - Alcance por grupo
    
Calcular margem de erro do índice
    

In [2]:
import cmath

In [12]:
alcance_promotor_top = 1000000 * (15/50)
alcance_detrator_top = 1000000 * (8/50)

In [13]:
alcance_promotor_res = 3000000 * (17/50)
alcance_detrator_res = 3000000 * (7/50)

In [20]:
alcance_promotor_total = alcance_promotor_top + alcance_promotor_res
alcance_detrator_total = alcance_detrator_top + alcance_detrator_res

In [23]:
indice_promotor_top = alcance_promotor_total / (alcance_promotor_top + alcance_detrator_top)
indice_promotor_res = alcance_promotor_total / (alcance_promotor_res+alcance_detrator_res)

In [24]:
indice_promotor_top

2.869565217391304

In [25]:
indice_promotor_res

0.9166666666666665

# Resposta Questão 1

A geração do índice depende do alcance de cada grupo. Para verificar a margem de erro do índice, a margem de erro dos grupos deve ser levado em consideração.

Utilizando a fórmula para descobrir o tamanho da amostra (https://pt.wikihow.com/Calcular-o-Tamanho-de-uma-Amostra) os dados de cada amostra foram utilizadas. (obs: 95% de confiança foi utilizado)

Para o grupo Top 50, como é um grupo controlado onde a população e amostra podem ser consideradas iguais, a margem de erro é 0. Já o restante, a marge de erro é cerca de 13% pois utilizo uma amostra de 50 de uma população de 550. Novamente, como o índice utiliza de ambos o grupo para compor o valor, essa margem será passada para o índice.

Se ao invés de segmentar utilizasse uma amostra de 100, a margem de erro seria menor (cerca de 8.95%) o que seria melhor para o índice.

Link de calculadoras:
- https://www.solvis.com.br/calculos-de-amostragem/
- https://pt.surveymonkey.com/mp/margin-of-error-calculator/
- https://www.opinionbox.com/calculadora-margem-de-erro/